In [25]:
#!/usr/bin/env python3
# mock_data_generator.py  ------------------------------------------
# สร้างไฟล์ SQL INSERT สำหรับ Airline Management DB
# ตรง schema 17‑Apr‑2025 – จำกัด VARCHAR(50) ตั้งแต่ต้น
# ------------------------------------------------------------------
import pandas as pd, random, math, json
from faker import Faker
from datetime import datetime, timedelta, date

fake = Faker()
Faker.seed(0); random.seed(0)

# ---------- util ---------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat, dlon = math.radians(lat2-lat1), math.radians(lon2-lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(a))

def to_sql(df, table):
    """แปลง DataFrame → INSERT (ไม่ตัด string อีก)"""
    cols = ", ".join(df.columns)
    values = []
    for _, r in df.iterrows():
        row = []
        for v in r:
            if pd.isnull(v):
                row.append("NULL")
            elif isinstance(v, str):
                row.append("'" + v.replace("'", "''") + "'")
            elif isinstance(v, bool):
                row.append(str(v).upper())
            elif isinstance(v, (datetime, date, pd.Timestamp)):
                row.append(f"'{v}'")
            else:
                row.append(str(v))
        values.append("(" + ", ".join(row) + ")")
    return f"INSERT INTO {table} ({cols}) VALUES\n" + ",\n".join(values) + ";\n"

# ---------- 1. airport (25) ---------------------------------------
airport_raw = [
    {"name":"Suvarnabhumi Airport","iata_code":"BKK","country":"Thailand","city":"Bangkok","latitude":13.69,"longitude":100.7501,"timezone":"Asia/Bangkok"},
    {"name":"Don Mueang International Airport","iata_code":"DMK","country":"Thailand","city":"Bangkok","latitude":13.9126,"longitude":100.6078,"timezone":"Asia/Bangkok"},
    {"name":"Phuket International Airport","iata_code":"HKT","country":"Thailand","city":"Phuket","latitude":8.1132,"longitude":98.3167,"timezone":"Asia/Bangkok"},
    {"name":"Chiang Mai International Airport","iata_code":"CNX","country":"Thailand","city":"Chiang Mai","latitude":18.7883,"longitude":98.9861,"timezone":"Asia/Bangkok"},
    {"name":"Haneda Airport","iata_code":"HND","country":"Japan","city":"Tokyo","latitude":35.5494,"longitude":139.7798,"timezone":"Asia/Tokyo"},
    {"name":"Narita International Airport","iata_code":"NRT","country":"Japan","city":"Tokyo","latitude":35.7767,"longitude":140.3189,"timezone":"Asia/Tokyo"},
    {"name":"Changi Airport","iata_code":"SIN","country":"Singapore","city":"Singapore","latitude":1.3644,"longitude":103.9915,"timezone":"Asia/Singapore"},
    {"name":"Heathrow Airport","iata_code":"LHR","country":"UK","city":"London","latitude":51.47,"longitude":-0.4543,"timezone":"Europe/London"},
    {"name":"Gatwick Airport","iata_code":"LGW","country":"UK","city":"London","latitude":51.1537,"longitude":-0.1821,"timezone":"Europe/London"},
    {"name":"Los Angeles International Airport","iata_code":"LAX","country":"USA","city":"Los Angeles","latitude":33.9416,"longitude":-118.4085,"timezone":"America/Los_Angeles"},
    {"name":"John F. Kennedy International Airport","iata_code":"JFK","country":"USA","city":"New York","latitude":40.6413,"longitude":-73.7781,"timezone":"America/New_York"},
    {"name":"San Francisco International Airport","iata_code":"SFO","country":"USA","city":"San Francisco","latitude":37.6213,"longitude":-122.379,"timezone":"America/Los_Angeles"},
    {"name":"Incheon International Airport","iata_code":"ICN","country":"South Korea","city":"Seoul","latitude":37.4602,"longitude":126.4407,"timezone":"Asia/Seoul"},
    {"name":"Dubai International Airport","iata_code":"DXB","country":"UAE","city":"Dubai","latitude":25.2532,"longitude":55.3657,"timezone":"Asia/Dubai"},
    {"name":"Frankfurt Airport","iata_code":"FRA","country":"Germany","city":"Frankfurt","latitude":50.1109,"longitude":8.6821,"timezone":"Europe/Berlin"},
    {"name":"Charles de Gaulle Airport","iata_code":"CDG","country":"France","city":"Paris","latitude":49.0097,"longitude":2.5479,"timezone":"Europe/Paris"},
    {"name":"Amsterdam Schiphol Airport","iata_code":"AMS","country":"Netherlands","city":"Amsterdam","latitude":52.3105,"longitude":4.7683,"timezone":"Europe/Amsterdam"},
    {"name":"Madrid‑Barajas Airport","iata_code":"MAD","country":"Spain","city":"Madrid","latitude":40.4936,"longitude":-3.5668,"timezone":"Europe/Madrid"},
    {"name":"Beijing Capital International Airport","iata_code":"PEK","country":"China","city":"Beijing","latitude":40.0801,"longitude":116.5846,"timezone":"Asia/Shanghai"},
    {"name":"Shanghai Pudong International Airport","iata_code":"PVG","country":"China","city":"Shanghai","latitude":31.1443,"longitude":121.8083,"timezone":"Asia/Shanghai"},
    {"name":"Sydney Airport","iata_code":"SYD","country":"Australia","city":"Sydney","latitude":-33.9399,"longitude":151.1753,"timezone":"Australia/Sydney"},
    {"name":"Auckland International Airport","iata_code":"AKL","country":"New Zealand","city":"Auckland","latitude":-37.0082,"longitude":174.7922,"timezone":"Pacific/Auckland"},
    {"name":"Istanbul Airport","iata_code":"IST","country":"Turkey","city":"Istanbul","latitude":41.2753,"longitude":28.7519,"timezone":"Europe/Istanbul"},
    {"name":"Sheremetyevo International Airport","iata_code":"SVO","country":"Russia","city":"Moscow","latitude":55.9726,"longitude":37.4146,"timezone":"Europe/Moscow"},
    {"name":"São Paulo/Guarulhos International Airport","iata_code":"GRU","country":"Brazil","city":"São Paulo","latitude":-23.4356,"longitude":-46.4731,"timezone":"America/Sao_Paulo"}
]
df_airport = pd.DataFrame(airport_raw).reset_index()
df_airport.rename(columns={'index':'airport_id'}, inplace=True)
df_airport['airport_id'] += 1

# ---------- 2. aircraft -------------------------------------------
models=[("Airbus A320",150,180),("Boeing 737",130,215),("Airbus A330",250,300),
        ("Boeing 777",300,400),("Boeing 787 Dreamliner",242,335),("Airbus A350",280,350)]
owners=["Thai Airways","Singapore Airlines","Emirates","American Airlines",
        "Lufthansa","Air France","Qantas","British Airways","Delta Air Lines"]
air=[]
for i in range(1,151):
    m,c1,c2=random.choice(models)
    air.append(dict(
        aircraft_id=i,
        model=m,
        manufacture_year=random.randint(2000,2024),
        capacity=random.randint(c1,c2),
        airline_owner=random.choice(owners),
        maintenance_status=random.choice(["Operational","In Maintenance","Retired"]),
        aircraft_history=fake.text(max_nb_chars=80)
    ))
df_aircraft = pd.DataFrame(air)

# ---------- 3. route ----------------------------------------------
routes=[]
for i in range(1,201):
    frm, to = random.sample(list(df_airport.airport_id), 2)
    a = df_airport.loc[df_airport.airport_id==frm].iloc[0]
    b = df_airport.loc[df_airport.airport_id==to].iloc[0]
    dist = round(haversine(a.latitude,a.longitude,b.latitude,b.longitude),2)
    routes.append(dict(
        route_id=i, from_airport=frm, to_airport=to,
        distance=dist,
        estimated_duration=str(timedelta(hours=dist/900))
    ))
df_route = pd.DataFrame(routes)

# ---------- 4. flight ---------------------------------------------
status_opt=["Scheduled","Boarding","Delayed","Cancelled","Completed"]
fl=[]
for i in range(1,2001):
    rid=random.randint(1,200)
    aid=random.randint(1,150)
    dep=fake.date_time_between("-90d","+90d")
    dur=pd.to_timedelta(df_route.loc[df_route.route_id==rid,'estimated_duration'].values[0])
    arr=dep+dur+timedelta(minutes=random.randint(-10,30))
    st=random.choices(status_opt,[70,10,10,5,5])[0]
    fl.append(dict(
        flight_id=i,
        flight_number=f"TG{random.randint(100,999)}",
        aircraft_id=aid, route_id=rid,
        departure_time=dep, arrival_time=arr,
        flight_status=st,
        cancellation_reason=fake.sentence() if st=="Cancelled" else None
    ))
df_flight = pd.DataFrame(fl)

# ---------- 5. crew -----------------------------------------------
crew_roles=["Pilot","Co-Pilot","Attendant","Technician"]
crew=[]
for i in range(1,501):
    crew.append(dict(
        crew_id=i,
        name=fake.name(),
        passport_number=fake.bothify("??######", letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")[:50],
        role=random.choice(crew_roles),
        license_expiry_date=fake.date_between("today","+5y"),
        passport_expiry_date=fake.date_between("today","+10y"),
        flight_hours=round(random.uniform(0,20000),2)
    ))
df_crew = pd.DataFrame(crew)

# ---------- 6. flight_crew_assignment -----------------------------
fca=[]
for f in range(1,2001):
    for c in random.sample(range(1,501), random.randint(3,6)):
        fca.append({"flight_id":f,"crew_id":c,"role_in_flight":random.choice(crew_roles)})
df_fca = pd.DataFrame(fca)

# ---------- 7. passenger ------------------------------------------
passengers=[]
for i in range(1,20001):
    fid=random.randint(1,2000)
    passengers.append(dict(
    passenger_id=i,
    name=fake.name(),
    passport_number=fake.bothify("??######",letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")[:50],
    nationality=fake.country()[:50],  # << ตัดความยาว nationality
    flight_id=fid,
    special_requests=fake.sentence()[:50] if random.random()<0.1 else None  # << ตัดตรงนี้ด้วย
))

df_passenger = pd.DataFrame(passengers)

# ---------- 8. ticket (seat unique) -------------------------------
t_status=["Booked","Cancelled","Used"]; c_status=["Pending","Checked-in"]
tickets=[]; seats_per_flight={}
for i in range(1,20001):
    fid=int(df_passenger.loc[i-1,'flight_id'])
    used=seats_per_flight.setdefault(fid,set())
    seat=None
    while seat is None or seat in used:
        seat=f"{random.randint(1,40)}{random.choice('ABCDEF')}"
    used.add(seat)
    tickets.append(dict(
        ticket_id=i,
        passenger_id=i,
        flight_id=fid,
        seat_number=seat[:50],
        ticket_status=random.choices(t_status,[80,5,15])[0],
        check_in_status=random.choice(c_status),
        booking_time=fake.date_time_between("-120d","now")
    ))
df_ticket = pd.DataFrame(tickets)

# ---------- 9. payment --------------------------------------------
methods=["Credit Card","PayPal","QR Code","Bank Transfer"]
payments=[]
for tid in random.sample(range(1,20001), int(20001*0.9)):
    payments.append(dict(
        payment_id=len(payments)+1,
        ticket_id=tid,
        payment_method=random.choice(methods)[:50],
        amount=round(random.uniform(100,1000),2),
        payment_date=fake.date_time_between("-90d","+90d"),
        payment_status=random.choice(["Pending","Completed","Failed","Refunded"])[:50],
        transaction_id=fake.uuid4()[:50]
    ))
df_payment = pd.DataFrame(payments)

# ---------- 10. maintenance_log -----------------------------------
maint=[]
for i in range(1,51):
    maint.append(dict(
        log_id=i,
        aircraft_id=random.randint(1,150),
        date_of_maintenance=fake.date_time_between("-1y","now"),
        details=fake.paragraph(nb_sentences=3)[:255],
        maintenance_location=fake.city()
    ))
df_maint = pd.DataFrame(maint)

# ---------- 11. users ---------------------------------------------
roles=["admin","crew","passenger","maintenance"]
users=[]
for i in range(1,1001):
    ct=fake.date_time_between("-2y","now")
    users.append(dict(
        user_id=i,
        username=fake.user_name()+str(i),
        hashed_password=fake.sha256(),
        email=None,
        role=random.choice(roles),
        is_active=True,
        last_login=None,
        created_at=ct,
        updated_at=fake.date_time_between(ct,"now")
    ))
df_users = pd.DataFrame(users)

# ---------- 12. audit_log -----------------------------------------
acts=["INSERT","UPDATE","DELETE"]
ents={"airport":25,"aircraft":150,"route":200,"flight":2000,
      "crew":500,"passenger":20000,"ticket":20000,
      "payment":len(df_payment),"users":1000}
audit=[]
for i in range(1,5001):
    ent=random.choice(list(ents))
    audit.append(dict(
        id=i,
        user_id=random.randint(1,1000),
        action=random.choice(acts)[:50],
        entity=ent[:50],
        entity_id=random.randint(1,ents[ent]),
        old_data=json.dumps({"field":fake.word(),"value":fake.word()}),
        new_data=json.dumps({"field":fake.word(),"value":fake.word()}),
        created_at=fake.date_time_between("-1y","now")
    ))
df_audit = pd.DataFrame(audit)

# ---------- write .sql files --------------------------------------
tables = {
    "airport": df_airport, "aircraft": df_aircraft, "route": df_route,
    "flight": df_flight, "crew": df_crew, "flight_crew_assignment": df_fca,
    "passenger": df_passenger, "ticket": df_ticket, "payment": df_payment,
    "maintenance_log": df_maint, "users": df_users, "audit_log": df_audit
}

for tbl, df in tables.items():
    with open(f"insert_{tbl}.sql","w",encoding="utf-8") as f:
        f.write(to_sql(df, tbl))

print("✅  SQL INSERT files generated for all 12 tables (string length safe).")


✅  SQL INSERT files generated for all 12 tables (string length safe).


In [24]:
# combine_mock_sql.py
# รวมไฟล์ insert_*.sql เป็นไฟล์เดียว mock_data_combined.sql
import os

# ลำดับไฟล์ตามความสัมพันธ์ Foreign Key
mock_sql_files = [
    "insert_airport.sql",
    "insert_aircraft.sql",
    "insert_route.sql",
    "insert_users.sql",
    "insert_flight.sql",
    "insert_crew.sql",
    "insert_flight_crew_assignment.sql",
    "insert_passenger.sql",
    "insert_ticket.sql",
    "insert_payment.sql",
    "insert_maintenance_log.sql",
    "insert_audit_log.sql"
]

# สร้างไฟล์รวม
combined_file = "mock_data_combined.sql"
with open(combined_file, "w", encoding="utf-8") as outfile:
    for filename in mock_sql_files:
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as infile:
                outfile.write(f"-- ===== {filename} =====\n")
                outfile.write(infile.read() + "\n")
        else:
            print(f"⚠️ Warning: {filename} not found!")

print(f"✅ รวมไฟล์ SQL สำเร็จ: {combined_file}")


✅ รวมไฟล์ SQL สำเร็จ: mock_data_combined.sql
